# Term Deposits Fundamentals

This notebook demonstrates how to access and use the main functionalities of **Term Deposits** within **LSEG Financial Analytics SDK**. 

**You will be able to:**
- Define term deposits instruments from a market instrument
- Configure pricing parameters
- Calculate Term Deposits analytics

## Imports

Import the following necessary modules:

- `lseg_analytics.pricing.instruments.term_deposit` - for Term Deposit instruments definitions and analytics

This notebook uses external library **pandas**; please ensure they are installed in your Python environment (e.g. 'pip install pandas') before running the code.

In [1]:
from lseg_analytics.pricing.instruments import term_deposit as td

import datetime as dt
import json
import pandas as pd

## Data Preparation

To define a Term Deposit instrument you need to follow a structured 3-step process:
1. **Term Deposit Definition** - Specify the term deposit instrument code (RIC)
2. **Term Deposit Instrument Definition** - Create the instrument object
3. **Pricing Preferences** - Configure pricing parameters, optional


In [2]:
# Create term deposit defintion object
term_deposit_definition = td.TermDepositDefinition(
    instrument_code = "EUR3MD=",
    notional_amount = 1000000
)

# Create term deposit instrument defintion object
term_deposit_instrument = td.TermDepositDefinitionInstrument(
    definition = term_deposit_definition
)

# Create term deposit pricing parameters object - optional
term_deposit_parameters = td.TermDepositPricingParameters(
    valuation_date  = dt.datetime.strptime("2025-07-21", "%Y-%m-%d"),
)


## Request Execution


In [3]:
#  Execute the calculation using the price() function
# The 'definitions' parameter accepts a list of instruments definitions for batch processing

term_deposit_response = td.price(
    definitions = [term_deposit_instrument], 
    pricing_preferences = term_deposit_parameters
)

## Results Display


#### Key Sections in the `response` JSON

 - **definitions**: Instrument setup (e.g., instrument code, fixed rate percent, start/end dates/tenors, notional, notional currency), it's TermDepositDefinition that we used. 

 - **pricingPreferences**: Valuation date, price side, reporting currency, market data date.

 - **analytics**:
   - **tabularData**: `data`, `headers`, `statuses`
   - **cashFlows**: gives details on the different cashflows of the term deposit product
   - **description**: Instrument summary TermDepositDefinition and also the default fields not specified in the TermDepositDefinition, but used by default in the calculation
   - **nominalMeasures**: different metrics on the term deposit product: average life, duration, modified duration, convexity, DV01
   - **pricingAnalysis**: `fixedRatePercent`, `valuationDate`, `marketDataDate`
   - **valuation**: `accruedDays`, `accruedAmountInDealCcy`, `marketValueInDealCcy`, `cleanMarketValueInDealCcy`
   - **error**: provides the root cause of the issue, if any

In [4]:
# Access the valuation object
valuation = term_deposit_response.data.analytics[0].valuation
print(json.dumps(valuation.as_dict(), indent=4))

{
    "accruedDays": 0,
    "accruedAmountInDealCcy": 0.0,
    "marketValueInDealCcy": 41.6889705392443,
    "cleanMarketValueInDealCcy": 41.6889705392443
}


In [5]:
# Convert to DataFrame
df_valuation = pd.DataFrame(list(valuation.items()), columns=["Fields", "Value"])
# Display the DataFrame
display(df_valuation)

,Fields,Value
0,accruedDays,0.000000
1,accruedAmountInDealCcy,0.000000
2,marketValueInDealCcy,41.688971
3,cleanMarketValueInDealCcy,41.688971


## Appendix: Complete Function Reference

### Available Classes and Functions in term_deposit Module

The following is a complete list of all classes, enums, and functions available after importing from `lseg_analytics.pricing.instruments.term_deposit`:

**Core Classes:**
- `TermDepositDefinition` - Define term deposit instrument specifications and parameters
- `TermDepositDefinitionInstrument` - Instrument container wrapper for pricing operations
- `TermDepositPricingParameters` - Configure pricing settings and market data preferences

**Response Classes:**
- `TermDepositCalculationResponse` - Main response container
- `TermDepositAnalyticsResponseData` - Detailed analytics and calculations
- `TermDepositAnalyticsResponseWithError` - Response with error

**Specific Term Deposit Classes within response:**
- `TermDepositDescription` - Gives the main characteristics of the instrument
- `TermDepositValuation` - Individual instrument valuation results
- `TermDepositPricingAnalysis` - Some information on pricing
- `TermDepositCashflows` - Detailed term deposit cash-flows
- `TermDepositNominalMeasures` - Different metrics on term deposit instrument

**Configuration Enums:**
- `InterestCalculationMethodEnum` - Day count conventions (DCB_ACTUAL_360, DCB_ACTUAL_365, etc.)
- `InterestPaymentFrequencyEnum` - Frequency of payments (ANNUAL, SEMI_ANNUAL, QUARTERLY, MONTHLY, etc.)
- `PaymentBusinessDayConventionEnum` - Business day conventions (NEXT_BUSINESS_DAY, MODIFIED_FOLLOWING, etc.)
- `PaymentRollConventionEnum` - Payment roll conventions (SAME, SAME28, LAST, LAST28, etc.)
- `PriceSideEnum` - Market price sides (BID, ASK, MID, LAST)

**Main Functions:**
- `price()` - Execute term deposit pricing calculations